# Extracción de Coeficientes PLP
Este notebook se enfoca en extraer los coeficientes PLP (Perceptual Linear Prediction) de una señal de audio. Los coeficientes PLP son comunes en el reconocimiento automático del habla y ofrecen una representación espectral perceptualmente ponderada de la señal.

## Índice:

1. Importación de Bibliotecas
2. Definición de Funciones para la Extracción de Coeficientes PLP
3. Cargar Datos de Audio
4. Extracción de Coeficientes PLP
5. Guardado de Coeficientes PLP
6. Resumen de los Resultados

## 1. Importación de Bibliotecas
Se importan las bibliotecas necesarias para la extracción y procesamiento de los coeficientes PLP.

In [ ]:
# Importamos las bibliotecas necesarias

# numpy es una biblioteca de Python utilizada para operaciones matemáticas
# sobre arrays y matrices de manera eficiente. Se abrevia comúnmente como np.
import numpy as np

# python_speech_features es una biblioteca que proporciona herramientas para
# extraer características de señales de audio, en particular características
# relacionadas con el habla. Estamos importando el módulo 'base' que contiene
# funciones fundamentales para la extracción de características.
from python_speech_features import base

## 2. Definición de Funciones para la Extracción de Coeficientes PLP
Se presentan las funciones que se usarán para cargar los datos de audio y extraer/guardar los coeficientes PLP.

### 2.1. Función para cargar frames de un archivo
Carga frames de audio desde un archivo utilizando la función `load` de numpy.

>**Parámetros:**
* **archivo_de_entrada (str):** Ruta o nombre del archivo del cual se cargarán los frames de audio.

>**Retorna:**
* **ndarray:** Un array de numpy con los frames de audio cargados desde el archivo.

In [ ]:
def load_frames_from_file(archivo_de_entrada):
    return np.load(archivo_de_entrada)

### 2.2. Funciones Auxiliares para Conversión y Filtros

1. **hz2bark:** Esta función convierte las frecuencias dadas en Hertz (Hz) a la escala Bark, que es una escala psicoacústica. Se basa en la observación empírica de la resolución del oído humano en diferentes frecuencias.

Convierte las frecuencias dadas de Hz a la escala Bark.

>**Parámetros:**
* **frequencies (ndarray):** Un array de numpy que contiene las frecuencias en Hz a convertir.

>**Retorna:**
* **ndarray:** Un array de numpy con las frecuencias convertidas a la escala Bark.

In [ ]:
def hz2bark(frequencies):
    return 6 * np.arcsinh(frequencies / 600)

2. **bark2hz:** Realiza la conversión opuesta a la función hz2bark, transformando las frecuencias en la escala Bark de vuelta a la escala de Hertz.

Convierte las frecuencias dadas de la escala Bark a Hz.

>**Parámetros:**
* **barks (ndarray):** Un array de numpy que contiene las frecuencias en la escala Bark a convertir.

>**Retorna:**
* **ndarray:** Un array de numpy con las frecuencias convertidas a Hz.

In [ ]:
def bark2hz(barks):
    return 600 * np.sinh(barks / 6)

3. **triangular_filters:** Esta función crea filtros triangulares basados en los bordes de las bandas críticas proporcionadas. Estos filtros se utilizan comúnmente en el procesamiento de señales de audio para analizar el contenido espectral del audio en bandas críticas específicas, que corresponden a la forma en que el oído humano percibe el sonido.

Genera filtros triangulares en el dominio de la frecuencia basados en los bordes de las bandas críticas proporcionadas.

>**Parámetros:**
* **critical_band_edges (ndarray):** Un array de numpy que contiene los bordes de las bandas críticas en Hz.
* **nfft (int):** El número total de puntos FFT.
* **sample_rate (int):** La tasa de muestreo del audio en Hz.

>**Retorna:**
* **ndarray:** Una matriz de numpy con los coeficientes de los filtros triangulares.

In [2]:
def triangular_filters(critical_band_edges, nfft, sample_rate):
    nfilt = len(critical_band_edges) - 2
    bin_indices = np.floor((nfft + 1) * critical_band_edges / sample_rate)
    fbanks = np.zeros((nfilt, nfft // 2 + 1))
    
    for i in range(nfilt):
        for j in range(int(bin_indices[i]), int(bin_indices[i+1])):
            fbanks[i, j] = (j - bin_indices[i]) / (bin_indices[i+1] - bin_indices[i])
        for j in range(int(bin_indices[i+1]), int(bin_indices[i+2])):
            fbanks[i, j] = (bin_indices[i+2] - j) / (bin_indices[i+2] - bin_indices[i+1])

    return fbanks

### 2.3. Función para extraer características PLP de los frames de audio
Extrae los coeficientes PLP (Perceptual Linear Prediction) de cada frame de audio.

>**Parámetros:**
* **frames (ndarray):** Una matriz donde cada columna representa un frame de audio.
* **sample_rate (int):** Tasa de muestreo del audio.
* **winstep (float, opcional):** Paso entre frames en segundos. Por defecto es 0.01 (10 ms).
* **numcep (int, opcional):** Número de coeficientes PLP a extraer. Por defecto es 13.
* **nfilt (int, opcional):** Número de filtros en el banco de filtros. Por defecto es 26.
* **nfft (int, opcional):** Número total de puntos FFT a usar. Por defecto es 512.

>**Retorna:**
* **ndarray:** Una matriz donde cada fila contiene los coeficientes PLP de un frame.

In [ ]:
def extraer_caracteristicas_plp_from_frames(frames, sample_rate, winstep=0.01, numcep=13, nfilt=26, nfft=512):
    frame_len = frames.shape[1]  # Cambiar a 1
    frame_width = frames.shape[0]  # Cambiar a 0

    plp_coeffs = np.zeros((frame_len, numcep))

    for i in range(frame_len):
        frame = frames[:, i]  # Cambiar a columna
        frame_power_spectrum = np.abs(np.fft.rfft(frame, n=nfft)) ** 2

        critical_band_edges = bark2hz(np.linspace(hz2bark(0), hz2bark(sample_rate // 2), nfilt + 2))
        fbanks = triangular_filters(critical_band_edges, nfft, sample_rate)

        # Aplicar el filtro bancario a la señal
        filter_banks_energy = np.dot(frame_power_spectrum, fbanks.T)
        filter_banks_energy = np.where(filter_banks_energy == 0, np.finfo(float).eps, filter_banks_energy)
        
        # Compresión de la señal
        filter_banks_energy = np.log(filter_banks_energy)

        # Obtener los coeficientes PLP
        plp_coeffs[i, :] = base.dct(filter_banks_energy)[:numcep]

    return plp_coeffs

## 3. Cargar Datos de Audio
Aquí se carga la señal de audio desde un archivo .npy.

In [ ]:
nombre_del_archivo_de_frames = "nombre_del_archivo_que_contiene_los_frames.npy"
frames = load_frames_from_file(nombre_del_archivo_de_frames)

## 4. Extracción de Coeficientes PLP
A partir de la señal de audio cargada, se extraen los coeficientes PLP.

In [ ]:
sample_rate = 32000
plp_coeffs = extraer_caracteristicas_plp_from_frames(frames, sample_rate)

## 5. Guardado de Coeficientes PLP
Una vez extraídos los coeficientes PLP, se procede a guardarlos en un archivo .npy.

In [ ]:
# Ruta al archivo donde se guardarán los coeficientes PLP
archivo_coeficientes = 'coeficientes_plp.npy'
np.save(archivo_coeficientes, plp_coeffs)

## 6. Resumen de los Resultados
1. Archivo original de frames: **nombre_del_archivo_que_contiene_los_frames.npy**
2. Archivo de salida con coeficientes PLP: **coeficientes_plp.npy**